In [11]:
class UpperAttrMetaclass(type):
    def __new__(cls, clsname, bases, attrs):
        uppercase_attrs = {
            attr if attr.startswith("__") else attr.upper(): v
            for attr, v in attrs.items()
        }

        # Python 3 can use no-arg super() which infers them:
        return super().__new__(cls, clsname, bases, uppercase_attrs)

class Foo(object, metaclass=UpperAttrMetaclass):
    hello = 'world'

    def show(self):
        print('hello foo!')

foo = Foo()

foo.SHOW()

hello foo!


In [42]:
def upper_attr(future_class_name, future_class_parents, future_class_attrs):
    """
      Return a class object, with the list of its attribute turned
      into uppercase.
    """
    print('future_class_name: {}'.format(future_class_name))
    print('future_class_parents: {}'.format(future_class_parents))
    print('future_class_attrs: {}'.format(future_class_attrs))

    uppercase_attrs = {
        attr if attr.startswith("__") else attr.upper(): v
        for attr, v in future_class_attrs.items()
    }

    return type(future_class_name, future_class_parents, uppercase_attrs)


class Bar(metaclass=upper_attr): 
    def __init__(self, name):
        self.name = name
    hello = 'world'

    def show(self):
        print('hello bar!')

bar = Bar('thu')

bar.SHOW()

future_class_name: Bar
future_class_parents: ()
future_class_attrs: {'__module__': '__main__', '__qualname__': 'Bar', '__init__': <function Bar.__init__ at 0x7fe2c5925550>, 'hello': 'world', 'show': <function Bar.show at 0x7fe2c588bdc0>}
hello bar!


In [18]:
bar.__dict__

{'name': 'thu'}

In [20]:
dict(a=1, b=2, c=3)

{'a': 1, 'b': 2, 'c': 3}

In [21]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

class Foo(metaclass=Meta):
    pass

Foo.attr

100

In [39]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        print(cls)
        print(name)
        print(bases)
        print(dct)

        dct = {**dct, **dict(attr=101)}
        
        print(dct)
        return super().__new__(cls, name, bases, dct)

class Foo(metaclass=Meta):
    hello = 'world'

Foo.attr

<class '__main__.Meta'>
Foo
()
{'__module__': '__main__', '__qualname__': 'Foo', 'hello': 'world'}
{'__module__': '__main__', '__qualname__': 'Foo', 'hello': 'world', 'attr': 101}


101

In [5]:
class Foo(object):

    def __new__(cls, *args, **kwargs):
        inst = object.__new__(cls, *args, **kwargs)
        print(cls)
        print(inst)
        return inst

    def __init__(self, price=50):
        self.price = price

    def how_much_of_book(self, n):
        print(self)
        return self.price * n

foo = Foo()
print(foo.how_much_of_book(8))


<class '__main__.Foo'>
400
